In [1]:
import sympy as sym
import numpy as np
from scipy import linalg

In [2]:
m = sym.nsimplify(0.5)
J1 = sym.nsimplify(0.0023)
J2 = sym.nsimplify(0.0023)
J3 = sym.nsimplify(0.0040)
l = sym.nsimplify(0.175)
g = sym.nsimplify(9.81)

In [3]:
# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# roll, pitch, yaw angles (radians)
phi, theta, psi = sym.symbols('phi, theta, psi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

# parameters
m = sym.nsimplify(0.5)
Jx = sym.nsimplify(0.0023)
Jy = sym.nsimplify(0.0023)
Jz = sym.nsimplify(0.0040)
l = sym.nsimplify(0.175)
g = sym.nsimplify(9.81)

# rotation matrices
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Rxyz = Rx * Ry * Rz

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry * Rz).T * ex, Rz.T * ey, ez).inv(), full=True)

# applied forces
forces = sym.Matrix([[0], [0], [-m * g]]) + Rxyz * sym.Matrix([[0], [0], [f_z]])

# euler's equations
euler = sym.Matrix([[(1 / Jx) * (tau_x + (Jy - Jz) * w_y * w_z)],
                   [(1 / Jy) * (tau_y + (Jz - Jx) * w_z * w_x)],
                   [(1 / Jz) * (tau_z + (Jx - Jy) * w_x * w_y)]])

# equations of motion
f = sym.Matrix.vstack(sym.Matrix([[v_x], [v_y], [v_z]]),
                      M * sym.Matrix([[w_x], [w_y], [w_z]]),
                      (1 / m) * forces,
                      euler)
f = sym.simplify(f, full=True)

In [4]:
f

Matrix([
[                                                     v_x],
[                                                     v_y],
[                                                     v_z],
[                (w_x*cos(psi) - w_y*sin(psi))/cos(theta)],
[                             w_x*sin(psi) + w_y*cos(psi)],
[-w_x*cos(psi)*tan(theta) + w_y*sin(psi)*tan(theta) + w_z],
[                                        2*f_z*sin(theta)],
[                              -2*f_z*sin(phi)*cos(theta)],
[                     2*f_z*cos(phi)*cos(theta) - 981/100],
[                          10000*tau_x/23 - 17*w_y*w_z/23],
[                          10000*tau_y/23 + 17*w_x*w_z/23],
[                                               250*tau_z]])

In [5]:
f_num = sym.lambdify((v_x, v_y, v_z,phi, theta, psi, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z), f)

In [6]:
v_x_e = 0
v_y_e = 0
v_z_e = 0
p_x_e = 0
p_y_e = 0
p_z_e = 0
phi_e = 0
theta_e = 0
psi_e = 0
w_x_e = 0
w_y_e = 0
w_z_e = 0
tau_x_e = 0
tau_y_e = 0
tau_z_e = 0
f_z_e = 9.81/2

In [7]:
f_num(v_x_e, v_y_e, v_z_e, phi_e, theta_e, psi_e, w_x_e, w_y_e, w_z_e, tau_x_e, tau_y_e, tau_z_e, f_z_e)

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [-0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [8]:
f_jacob_x = f.jacobian([p_x, p_y, p_z, phi, theta, psi, v_x, v_y, v_z, w_x, w_y, w_z])
f_jacob_u = f.jacobian([tau_x, tau_y, tau_z, f_z])

In [9]:
A_num = sym.lambdify((p_x, p_y, p_z, phi, theta, psi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z),f_jacob_x)
B_num = sym.lambdify((p_x, p_y, p_z, phi, theta, psi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z),f_jacob_u)

A = A_num(p_x_e, p_y_e, p_z_e, phi_e, theta_e, psi_e, v_x_e, v_y_e, v_z_e, w_x_e, w_y_e, w_z_e, tau_x_e, tau_y_e, tau_z_e, f_z_e)
B = B_num(p_x_e, p_y_e, p_z_e, phi_e, theta_e, psi_e, v_x_e, v_y_e, v_z_e, w_x_e, w_y_e, w_z_e, tau_x_e, tau_y_e, tau_z_e, f_z_e)

In [10]:
A.tolist()

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, -9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [11]:
B.tolist()

[[0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, -0.0],
 [0.0, 0.0, 0.0, 2.0],
 [434.7826086956522, 0.0, 0.0, 0.0],
 [0.0, 434.7826086956522, 0.0, 0.0],
 [0.0, 0.0, 250.0, 0.0]]

In [12]:
## Checking for controllability
# Find the number of states
n = A.shape[0]

# Initialize W with its first column
Wc = B

# Create W one column at a time by iterating over i from 1 to n-1
for i in range(1, n):
    col = np.linalg.matrix_power(A, i) @ B
    Wc = np.block([Wc, col])

In [13]:
n

12

In [14]:
# What about the rank?
print(np.linalg.matrix_rank(Wc))

12


In [15]:
C = np.array([[1,0,0,0,0,0,0,0,0,0,0,0],
              [0,1,0,0,0,0,0,0,0,0,0,0],
              [0,0,1,0,0,0,0,0,0,0,0,0],
              [0,0,0,1,0,0,0,0,0,0,0,0],
              [0,0,0,0,1,0,0,0,0,0,0,0],
              [0,0,0,0,0,1,0,0,0,0,0,0]])

In [16]:
# We can check for the observability of this system
Wo = C
for i in range(1,n):
    row = C @ np.linalg.matrix_power(A,i)
    Wo = np.block([[Wo],[row]])
rank = np.linalg.matrix_rank(Wo)
print(rank)
# The rank of the system is the same as the number of states, so the system is observable.

12


In [17]:
Qc = np.diag([5.,25.,5.,24.,24.,24.,4.,8.,4.,8.,8.,8.]) 
Rc = np.diag([70.,70.,70.,2.])
# We can also define an LQR function in the following way:
def lqr(A,B,Q,R):
    P = linalg.solve_continuous_are(A,B,Q,R)
    K = linalg.inv(R) @ B.T @ P
    return K
# The function above can be used for both controller and observer design

# Designing an optimal observer
Qo = 100*np.diag(np.ones(6))
Ro = np.diag(np.ones(12))

L = lqr(A.T, C.T, linalg.inv(Ro), linalg.inv(Qo)).T

K = lqr(A,B,Qc,Rc)

In [18]:
L.tolist()

[[11.3058803110068, 0.0, 0.0, 0.0, 0.4396776833728971, 0.0],
 [0.0, 11.3058803110068, 0.0, -0.4396776833728971, 0.0, 0.0],
 [0.0, 0.0, 10.954451150103324, 0.0, 0.0, 0.0],
 [0.0, -0.4396776833728971, 0.0, 10.945603950560619, 0.0, 0.0],
 [0.4396776833728971, 0.0, 0.0, 0.0, 10.945603950560619, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 10.954451150103324],
 [14.00812303603352, 0.0, 0.0, 0.0, 9.717323102417483, 0.0],
 [0.0, 14.00812303603352, 0.0, -9.717323102417483, 0.0, 0.0],
 [0.0, 0.0, 10.0, 0.0, 0.0, 0.0],
 [0.0, -0.06615794931719185, 0.0, 9.999781153892402, 0.0, 0.0],
 [0.06615794931719185, 0.0, 0.0, 0.0, 9.999781153892402, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 10.0]]

In [19]:
K.tolist()

[[-7.470103824860964e-16,
  -0.597614304667193,
  -2.534736450492715e-16,
  2.131279246713305,
  -2.2795088647501232e-15,
  1.2174213473461131e-14,
  -5.725766455299804e-16,
  -0.6115194547658369,
  2.1235858790732371e-16,
  0.3522635360360125,
  1.882304521121369e-17,
  9.777282575744851e-17],
 [0.26726124191242473,
  -2.9636468334901435e-17,
  3.3383667491262037e-16,
  -4.687289900146584e-16,
  1.7324268494946549,
  4.0340005770852824e-17,
  0.3892794424612655,
  4.4031235194257394e-16,
  3.996567687509206e-18,
  1.882304521121369e-17,
  0.3496496500690224,
  -3.7084882309639604e-17],
 [-2.4083942781012557e-16,
  -9.452569554399674e-15,
  -1.6235935707886185e-16,
  9.560492828896679e-15,
  7.267272416605851e-16,
  0.5855400437691188,
  1.0873236001454141e-16,
  -3.634259171884253e-15,
  -9.553253349797393e-17,
  5.621937481053289e-17,
  -2.1323807328042768e-17,
  0.34492033085318],
 [-8.423703219100551e-17,
  -1.1812055222399395e-15,
  1.581138830084189,
  1.754554067453382e-15,
  -7

In [23]:
F = A - L@C
s = linalg.eigvals(F)
s

array([-4.09300073+0.j        , -2.131011  +1.84879596j,
       -2.131011  -1.84879596j, -4.09300073+0.j        ,
       -2.131011  +1.84879596j, -2.131011  -1.84879596j,
       -1.00051096+0.j        , -1.00051096+0.j        ,
       -2.97875534+0.j        , -2.97875534+0.j        ,
       -1.06161041+0.j        , -1.06161041+0.j        ])